In [ ]:
import time
import math, os

from coffea.nanoevents import BaseSchema

import awkward as ak
import numpy as np
from coffea import processor

from hist import Hist

from coffea.nanoevents.methods import candidate
ak.behavior.update(candidate.behavior)

import matplotlib.pyplot as plt

In [ ]:
## Particle collections

# Gen particle collections
gen_part_cols = ['GenPart_eta', 'GenPart_genPartIdxMother', 'GenPart_mass', 'GenPart_pdgId', "GenPart_phi", "GenPart_pt", 'GenPart_status',
                 'GenPart_Id', 'GenPart_parpdgId', 'GenPart_sparpdgId', 'GenPart_numberOfDaughters', 'GenPart_nstchgdaug', 'GenPart_vx', 
                 'GenPart_vy', 'GenPart_vz', 'GenPart_mvx', 'GenPart_mvy', 'GenPart_mvz', 'GenPart_recIdx']

# Muon collections
muon_cols = ['Muon_charge', 'Muon_dxy', 'Muon_dxyErr', 'Muon_dz', 'Muon_dzErr', 'Muon_eta', 'Muon_isGlobal', 'Muon_mass',
             'Muon_phi', 'Muon_pt', 'Muon_ptErr', 'Muon_softId', 'Muon_vtxIdx', 'Muon_vtxFlag', 'Muon_simIdx']

# Dimuon collections
dimu_cols = ['Dimu_pt', 'Dimu_eta', 'Dimu_phi', 'Dimu_rap', 'Dimu_mass', 'Dimu_charge', 'Dimu_vtxIdx', 'Dimu_chi2', 'Dimu_dl',
             'Dimu_dlErr', 'Dimu_dlSig', 'Dimu_cosphi', 'Dimu_x', 'Dimu_y', 'Dimu_z', 'Dimu_t1muIdx', 'Dimu_t2muIdx',]


# Function to get the collections
def get_vars_dict(events, col_list):
    dict = {}
    col = ''
    for c in col_list:
        if c.startswith('Muon'):
            col = c[5:]
        elif c.startswith('Dimu'):
            col = c[4:]
            if col.startswith('_'): col = col[1:]
        elif c.startswith('D0'):
            col = c[2:]
            if col.startswith('_'): col = col[1:]
        elif c.startswith('Dstar'):
            col = c[5:]
            if col.startswith('_'): col = col[1:]
        elif c.startswith('PVtx'):
            col = c[5:]
        elif c.startswith("GenPart"):
            col = c[8:]
        else:
            Exception('Not good!')

        if col == 'x' or col == 'y' or col == 'z':
            col = 'vtx_' + col

        if len(events[c]) == 0:
            dict[col] = np.array([])
        else:
            dict[col] = events[c]
    return dict

In [ ]:
import re

files = []
# Uses scandir to iterate over the root files
with os.scandir("/eos/user/m/mabarros/Monte_Carlo/jpsi_gun/0000") as it:
    # Iterates over the files
    for file in it:
        if file.name.endswith('.root') and (file.stat().st_size != 0):
            # Stores the file
            files.append(file.path)

## Code block to sort file names
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]
            
files.sort(key=natural_keys)

In [ ]:
import matplotlib.pyplot as plt
import mplhep
from matplotlib.text import Text
from hist.intervals import ratio_uncertainty

plt.style.use(mplhep.style.CMS)
    
plt.rcParams.update({
    'font.size': 16,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14
})

In [ ]:
def acceptance_plot(hist_reco, hist_gen, ax=None, with_labels=True):

    ratio = hist_reco / hist_gen.values()

    if with_labels:
        err_down, err_up = ratio_uncertainty(hist_reco.values(), hist_gen.values())
        labels = []
        for ra, u, d in zip(ratio.values().ravel(), err_up.ravel(), err_down.ravel()):
            ra, u, d = f'{ra:.2f}', f'{u:.2f}', f'{d:.2f}'
            st = '$'+ra+'_{-'+d+'}^{+'+u+'}$'
            labels.append(st)
        labels = np.array(labels).reshape(10,10)
        
        mplhep.hist2dplot(ratio, labels=labels, ax=ax)
        # CMS format
        hfont = {'fontname':'Helvetica'}    
        plt.text(0.13, 0.89, "CMS", fontdict=hfont,  fontweight='bold', transform=plt.gcf().transFigure)
        plt.text(0.17, 0.89, "Simulation", fontdict=hfont, style='italic',fontsize = 100, transform=plt.gcf().transFigure)

        x = ax.get_children()
        for i0 in x:
            if isinstance(i0, Text):
                i0.set_size(10)

        return ax
    else:
        mplhep.hist2dplot(ratio, ax=ax)
        return ax

In [ ]:
class AcceptanceProcessor(processor.ProcessorABC):
    def __init__(self):
        Gen_Muon_hists = {
            'Gen_Muon_pt': Hist.new.Reg(100, 0, 100, name='pt', label=r"$p_{T,\mu}$ [GeV]").Double(),
            'Gen_Muon_eta': Hist.new.Reg(60, -4, 4, name="eta", label=r"$\eta_{\mu}$").Double(),
            'Gen_Muon_phi': Hist.new.Reg(60, -math.pi, math.pi, name="phi", label=r"$\phi_{\mu}$").Double(),
            
            'Gen_Muon_pt_0to100_x_eta-4to4': (
                Hist.new
                .Reg(10, 0, 100, name='pt', label=r"$p_{T,\mu^+\mu^-}$ [GeV]")
                .Reg(10, -4, 4, name="eta", label=r"$\eta_{\mu^+\mu^-}$")
                .Double()
                ),
            
            'Gen_Muon_pt_0to100_x_eta-2p5to2p5': (
                Hist.new
                .Reg(10, 0, 100, name='pt', label=r"$p_{T,\mu^+\mu^-}$ [GeV]")
                .Reg(10, -2.5, 2.5, name="eta", label=r"$\eta_{\mu^+\mu^-}$")
                .Double()
                ),
            
            'Gen_Muon_pt_0to10_x_eta-2p5to2p5': (
                Hist.new
                .Reg(10, 0, 10, name='pt', label=r"$p_{T,\mu^+\mu^-}$ [GeV]")
                .Reg(10, -2.5, 2.5, name="eta", label=r"$\eta_{\mu^+\mu^-}$")
                .Double()
                ), 
            
            'Gen_Muon_pt_0to10_x_eta0to2p5': (
                Hist.new
                .Reg(10, 0, 10, name='pt', label=r"$p_{T,\mu^+\mu^-}$ [GeV]")
                .Reg(10, 0, 2.5, name="eta", label=r"$|\eta_{\mu^+\mu^-}|$")
                .Double()
                ),
            'Gen_Muon_pt_0to30_x_eta0to2p5': (
                Hist.new
                .Reg(10, 0, 2.5, name="eta", label=r"$|\eta_{\mu^+\mu^-}|$")
                .Reg(10, 0, 30, name='pt', label=r"$p_{T,\mu^+\mu^-}$ [GeV]")
                .Double()
                ), 
            
        }

        Reco_Muon_hists = {
            'Reco_Muon_pt': Hist.new.Reg(100, 0, 100, name='pt', label=r"$p_{T,\mu^+\mu^-}$ [GeV]").Double(),
            'Reco_Muon_eta': Hist.new.Reg(60, -4, 4, name="eta", label=r"$\eta_{\mu^+\mu^-}$").Double(),
            'Reco_Muon_phi': Hist.new.Reg(60, -math.pi, math.pi, name="phi", label=r"$\phi_{\mu^+\mu^-}$").Double(),
           
            'Reco_Muon_pt_0to100_x_eta-4to4': (
                Hist.new
                .Reg(10, 0, 100, name='pt', label=r"$p_{T,\mu}$ [GeV]")
                .Reg(10, -4, 4, name="eta", label=r"$\eta_{\mu}$")
                .Double()
                ),
            'Reco_Muon_pt_0to100_x_eta-2p5to2p5': (
                Hist.new
                .Reg(10, 0, 100, name='pt', label=r"$p_{T,\mu}$ [GeV]")
                .Reg(10, -2.5, 2.5, name="eta", label=r"$\eta_{\mu}$")
                .Double()
                ),
            'Reco_Muon_pt_0to10_x_eta-2p5to2p5': (
                Hist.new
                .Reg(10, 0, 10, name='pt', label=r"$p_{T,\mu}$ [GeV]")
                .Reg(10, -2.5, 2.5, name="eta", label=r"$\eta_{\mu}$")
                .Double()
                ),
            'Reco_Muon_pt_0to10_x_eta0to2p5': (
                Hist.new
                .Reg(10, 0, 10, name='pt', label=r"$p_{T,\mu}$ [GeV]")
                .Reg(10, 0, 2.5, name="eta", label=r"$|\eta_{\mu}|$")
                .Double()
                ),
            'Reco_Muon_pt_0to30_x_eta0to2p5': (
                Hist.new
                .Reg(10, 0, 2.5, name="eta", label=r"$|\eta_{\mu}|$")
                .Reg(10, 0, 30, name='pt', label=r"$p_{T,\mu}$ [GeV]")
                .Double()
                ),
        }
        
        self._hists = {
            'Gen_Muon': {**Gen_Muon_hists},
            'Reco_Muon': {**Reco_Muon_hists},
        }
        
        self._accumulator = processor.dict_accumulator({
            'details': processor.defaultdict_accumulator(int),
        })
        
        
    @property
    def accumulator(self):
        return self._accumulator
    
    @property
    def hists(self):
        return self._hists

    def process(self, events):
        output = self.accumulator.identity()
        
        # test if there is any events in the file
        if len(events) == 0:
            return output

        # Collection extraction
        GenPart = ak.zip({**get_vars_dict(events, gen_part_cols)}, with_name="PtEtaPhiMCandidate")
        
        # Dimuon collection
        Dimus = ak.zip({**get_vars_dict(events, dimu_cols)}, with_name="PtEtaPhiMCandidate")
        # Muons collection
        Muons = ak.zip({**get_vars_dict(events, muon_cols)}, with_name="PtEtaPhiMCandidate")
        
        ## Rec dimuon cuts
        
        #Dimu cuts charge = 0, mass cuts and chi2...
        Dimu = Dimus[Dimus.charge == 0]

        Dimu = Dimu[(Dimu.mass > 2.95) & (Dimu.mass < 3.25)]

        # Get the Muons from Dimu, for cuts in their params
        Muon = ak.zip({'0': Muons[Dimu.t1muIdx], '1': Muons[Dimu.t2muIdx]})
       
        # Takes all gen muons (Some decays from pdgId: -411, 421, 511, 521, 431, 4132)
        gen_muon = GenPart[(np.absolute(GenPart.pdgId) == 13) & (GenPart.genPartIdxMother > -1)]
        # Takes gen muons from Jpis
        gen_muon = gen_muon[(gen_muon.parpdgId == 443)]
        # Combines muons two by two
        gen_muon = ak.combinations(gen_muon, 2)
        # Takes combinations with the same mother
        gen_muon = gen_muon[gen_muon.slot0.genPartIdxMother == gen_muon.slot1.genPartIdxMother]
        
        # Combine each gen particle with each muon
        comb_gen_muon = ak.cartesian([GenPart, gen_muon.slot0])
        comb_dimuon_muon = comb_gen_muon[comb_gen_muon.slot0.Id == comb_gen_muon.slot1.genPartIdxMother]
        gen_dimu = comb_dimuon_muon.slot0  
        
        # Gen muons
        gen_muon1 = gen_muon.slot0
        gen_muon2 = gen_muon.slot1
        
        # Rec muons
        reco_muon1 = Muon.slot0
        reco_muon2 = Muon.slot1
        
        ## Hist: from 0 to 100 GeV and from -4 to 4 in eta    
        self.hists['Gen_Muon']['Gen_Muon_pt_0to100_x_eta-4to4'].fill(
            pt=ak.flatten(gen_muon1.pt),
            eta=ak.flatten(gen_muon1.eta)
        )
        self.hists['Gen_Muon']['Gen_Muon_pt_0to100_x_eta-4to4'].fill(
            pt=ak.flatten(gen_muon2.pt),
            eta=ak.flatten(gen_muon2.eta)
        )
        
        self.hists['Reco_Muon']['Reco_Muon_pt_0to100_x_eta-4to4'].fill(
            pt=ak.flatten(reco_muon1.pt),
            eta=ak.flatten(reco_muon1.eta)
        )
        self.hists['Reco_Muon']['Reco_Muon_pt_0to100_x_eta-4to4'].fill(
            pt=ak.flatten(reco_muon2.pt),
            eta=ak.flatten(reco_muon2.eta)
        )
        
        ## Hist: from 0 to 100 GeV and from -2.5 to 2.5 in eta    
        self.hists['Gen_Muon']['Gen_Muon_pt_0to100_x_eta-2p5to2p5'].fill(
            pt=ak.flatten(gen_muon1.pt),
            eta=ak.flatten(gen_muon1.eta)
        )
        self.hists['Gen_Muon']['Gen_Muon_pt_0to100_x_eta-2p5to2p5'].fill(
            pt=ak.flatten(gen_muon2.pt),
            eta=ak.flatten(gen_muon2.eta)
        )
        
        self.hists['Reco_Muon']['Reco_Muon_pt_0to100_x_eta-2p5to2p5'].fill(
            pt=ak.flatten(reco_muon1.pt),
            eta=ak.flatten(reco_muon1.eta)
        )
        self.hists['Reco_Muon']['Reco_Muon_pt_0to100_x_eta-2p5to2p5'].fill(
            pt=ak.flatten(reco_muon2.pt),
            eta=ak.flatten(reco_muon2.eta)
        )
        
        ## Hist: from 0 to 10 GeV and from -2.5 to 2.5 in eta 
        self.hists['Gen_Muon']['Gen_Muon_pt_0to10_x_eta-2p5to2p5'].fill(
            pt=ak.flatten(gen_muon1.pt),
            eta=ak.flatten(gen_muon1.eta)
        )
        self.hists['Gen_Muon']['Gen_Muon_pt_0to10_x_eta-2p5to2p5'].fill(
            pt=ak.flatten(gen_muon2.pt),
            eta=ak.flatten(gen_muon2.eta)
        )
        self.hists['Reco_Muon']['Reco_Muon_pt_0to10_x_eta-2p5to2p5'].fill(
            pt=ak.flatten(reco_muon1.pt),
            eta=ak.flatten(reco_muon1.eta)
        )
        self.hists['Reco_Muon']['Reco_Muon_pt_0to10_x_eta-2p5to2p5'].fill(
            pt=ak.flatten(reco_muon2.pt),
            eta=ak.flatten(reco_muon2.eta)
        )
        
        ## Hist: from 0 to 10 GeV and from 0 to 2.5 in eta 
        self.hists['Gen_Muon']['Gen_Muon_pt_0to10_x_eta0to2p5'].fill(
            pt=ak.flatten(gen_muon1.pt),
            eta=ak.flatten(np.absolute(gen_muon1.eta))
        )
        self.hists['Gen_Muon']['Gen_Muon_pt_0to10_x_eta0to2p5'].fill(
            pt=ak.flatten(gen_muon2.pt),
            eta=ak.flatten(np.absolute(gen_muon2.eta))
        )
        self.hists['Reco_Muon']['Reco_Muon_pt_0to10_x_eta0to2p5'].fill(
            pt=ak.flatten(reco_muon1.pt),
            eta=ak.flatten(np.absolute(reco_muon1.eta))
        )
        self.hists['Reco_Muon']['Reco_Muon_pt_0to10_x_eta0to2p5'].fill(
            pt=ak.flatten(reco_muon2.pt),
            eta=ak.flatten(np.absolute(reco_muon2.eta))
        )
        
        ## Hist: from 0 to 30 GeV and from 0 to 2.5 in eta 
        self.hists['Gen_Muon']['Gen_Muon_pt_0to30_x_eta0to2p5'].fill(
            pt=ak.flatten(gen_muon1.pt),
            eta=ak.flatten(np.absolute(gen_muon1.eta))
        )
        self.hists['Gen_Muon']['Gen_Muon_pt_0to30_x_eta0to2p5'].fill(
            pt=ak.flatten(gen_muon2.pt),
            eta=ak.flatten(np.absolute(gen_muon2.eta))
        )
        self.hists['Reco_Muon']['Reco_Muon_pt_0to30_x_eta0to2p5'].fill(
            pt=ak.flatten(reco_muon1.pt),
            eta=ak.flatten(np.absolute(reco_muon1.eta))
        )
        self.hists['Reco_Muon']['Reco_Muon_pt_0to30_x_eta0to2p5'].fill(
            pt=ak.flatten(reco_muon2.pt),
            eta=ak.flatten(np.absolute(reco_muon2.eta))
        )
        
    
        return self.hists

    def postprocess(self, accumulator):
        return accumulator

In [ ]:
data = {"test": files[:150]}

tstart = time.time()

output = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=AcceptanceProcessor(),
                                    executor=processor.futures_executor,
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

""" hists = output[1]
output = output[0] """

print(f"Process finished in: {time.time() - tstart:.2f} s")
#print(output[0]['details'])

In [ ]:
fig = plt.figure()
fig.clear()
ax = fig.add_subplot()
acceptance_plot(output['Reco_Muon']['Reco_Muon_pt_0to100_x_eta-4to4'], output['Gen_Muon']['Gen_Muon_pt_0to100_x_eta-4to4'], ax=ax)

In [ ]:
fig = plt.figure()
fig.clear()
ax = fig.add_subplot()
acceptance_plot(output['Reco_Muon']['Reco_Muon_pt_0to100_x_eta-2p5to2p5'], output['Gen_Muon']['Gen_Muon_pt_0to100_x_eta-2p5to2p5'], ax=ax)

In [ ]:
fig = plt.figure()
fig.clear()
ax = fig.add_subplot()
acceptance_plot(output['Reco_Muon']['Reco_Muon_pt_0to10_x_eta-2p5to2p5'], output['Gen_Muon']['Gen_Muon_pt_0to10_x_eta-2p5to2p5'], ax=ax)

In [ ]:
fig = plt.figure()
fig.clear()
ax = fig.add_subplot()
acceptance_plot(output['Reco_Muon']['Reco_Muon_pt_0to10_x_eta0to2p5'], output['Gen_Muon']['Gen_Muon_pt_0to10_x_eta0to2p5'], ax=ax)

In [ ]:
fig = plt.figure()
fig.clear()
ax = fig.add_subplot()
acceptance_plot(output['Reco_Muon']['Reco_Muon_pt_0to30_x_eta0to2p5'], output['Gen_Muon']['Gen_Muon_pt_0to30_x_eta0to2p5'], ax=ax)